<a href="https://colab.research.google.com/github/nikitanguturi/heinwebdriver/blob/main/HeinWebDriver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Section 0: User Definition

In [1]:
import pandas as pd

# User Data
current_user = "Christos"
if current_user == "Christos":
  # Define input and output path
  author_data_path = "/content/drive/MyDrive/Hein Scrapper/authorTestSheet.csv"
  output_path = "/content/drive/MyDrive/Hein Scrapper/heinonline_authors_metrics.csv"

  # Load author data into a DataFrame
  author_data = pd.read_csv(author_data_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Hein Scrapper/authorTestSheet.csv'

### Step 1: Libray Set Up


In [ ]:
# Installs
# Install selenium and other dependencies
!pip install selenium
!pip install beautifulsoup4
!pip install pandas

In [ ]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

#Step 2: Web Driver Creation and Initlialization
This section creates the webdriver a directs it to the log-in page and has it log on.

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# Navigate to HeinOnline login page
login_url = "https://heinonline-org.ezproxy.bu.edu/login"
driver.get(login_url)

# Locate login fields and enter credentials
username_field = driver.find_element(By.ID, "username")  # Adjust based on actual HTML structure
password_field = driver.find_element(By.ID, "password")  # Adjust based on actual HTML structure

username_field.send_keys("Nrt03")
password_field.send_keys("B@urdrillard")
password_field.send_keys(Keys.RETURN)  # Press Enter to submit

# Wait for the login process to complete
time.sleep(3)  # Adjust based on loading time

# Optional: Store cookies for session management if needed
cookies = driver.get_cookies()

# Step 3: Construct URL and Cycle through Authors
This section formats the URL for each author with the values from the Excel file. It then goes to eahc authors page and collects the HTML data from each author's page while varying headers and time intervals between requests.

In [ ]:
# Initialize a DataFrame to store author data and metrics
columns = [
    "FirstName", "LastName",
    "Cited Articles (0-5 Years)", "Cited Articles (5+ Years)",
    "Cited Cases (0-5 Years)", "Cited Cases (5+ Years)",
    "Accessed (Past 12 Months)", "ScholarRank",
    "Average Citations per Article", "Average Citations per Document",
    "Self-Citations", "H-Index"
]
result_data = pd.DataFrame(columns=columns)

# Helper function to extract metric based on the <th> text
def get_metric(soup, metric_name):
    metric_row = soup.find("th", string=metric_name)
    if metric_row:
        metric_value = metric_row.find_next("td").text.strip()
        return metric_value
    return "N/A"

# Iterate through each author and load their profile
for index, row in author_data.iterrows():
    last_name = row["LastName"]
    first_name = row["FirstName"]
    author_url = base_url.format(lastName=last_name, firstName=first_name)

    # Navigate to the author's page
    driver.get(author_url)
    time.sleep(random.uniform(2, 5))  # Add a delay to mimic natural browsing

    # Get the HTML content and parse with BeautifulSoup
    page_html = driver.page_source
    soup = BeautifulSoup(page_html, 'html.parser')

    # Extract each metric using the helper function
    cited_articles_0_5 = get_metric(soup, "Cited by Articles (0-5 Years)")
    cited_articles_5_plus = get_metric(soup, "Cited by Articles (5+ Years)")
    cited_cases_0_5 = get_metric(soup, "Cited by Cases (0-5 Years)")
    cited_cases_5_plus = get_metric(soup, "Cited by Cases (5+ Years)")
    accessed_past_12_months = get_metric(soup, "Accessed (Past 12 Months)")
    scholar_rank = get_metric(soup, "ScholarRank")
    avg_citations_per_article = get_metric(soup, "Average Citations per Article")
    avg_citations_per_doc = get_metric(soup, "Average Citations per Document")
    self_citations = get_metric(soup, "Self-Citations")
    h_index = get_metric(soup, "H-Index")

    # Append the data for this author into the DataFrame
    result_data = result_data.append({
        "FirstName": first_name,
        "LastName": last_name,
        "Cited Articles (0-5 Years)": cited_articles_0_5,
        "Cited Articles (5+ Years)": cited_articles_5_plus,
        "Cited Cases (0-5 Years)": cited_cases_0_5,
        "Cited Cases (5+ Years)": cited_cases_5_plus,
        "Accessed (Past 12 Months)": accessed_past_12_months,
        "ScholarRank": scholar_rank,
        "Average Citations per Article": avg_citations_per_article,
        "Average Citations per Document": avg_citations_per_doc,
        "Self-Citations": self_citations,
        "H-Index": h_index
    }, ignore_index=True)

# Save the final DataFrame to Google Drive
result_data.to_csv(output_path, index=False)